In [17]:
import xarray as xr
import pandas as pd
import numpy as np

In [18]:
export_folder = "/Users/ash/Documents/Workspaces/Data/phd/SEAPODYM_LMTL/2024-12-13_-_Constant_fields_no_transport/data"

My data is not working in BIOMASS computation process with LMTL for now.


In [19]:
# bio = xr.load_dataset(
#     "/Users/ash/Documents/Workspaces/Data/cmems/2024-07 - 2x2 - LMTL no transport/cmems_mod_glo_bgc_my_0.083deg-lmtl_PT1D-i_1723016267080.nc"
# )
# physic = xr.load_dataset(
#     "/Users/ash/Documents/Workspaces/Data/cmems/2024-07 - 2x2 - LMTL no transport/cmems_mod_glo_bgc_my_0.083deg-lmtl-Fphy_PT1D-i_1723015794520.nc"
# )
# data = xr.merge([bio, physic])[["T", "U", "V", "npp"]]
# attrs = data["depth"].attrs
# data = data.assign_coords({"depth": [0, 1, 2]})
# data.depth.attrs = attrs
# data

Instead I use the forcing data from MERCATOR and set the values to constants.


In [ ]:
data = xr.open_mfdataset(
    "/Users/ash/Documents/Workspaces/Data/configurations_seapodym/lmtl/4_data_integrated/4_2_epi_upmeso_daily/*.nc"
).load()
data = data.rename({"u": "U", "v": "V", "temperature": "T"})
data = data[["T", "U", "V", "npp"]]
data = data.resample(time="D").mean()


attrs = data["U"].attrs
data["U"] = xr.full_like(data["U"], 0)
data["U"].attrs = attrs

attrs = data["V"].attrs
data["V"] = xr.full_like(data["V"], 0)
data["V"].attrs = attrs

attrs = data["T"].attrs
TEMPERATURE = 28
data["T"] = xr.full_like(data["T"], TEMPERATURE)
data["T"].attrs = attrs

attrs = data["npp"].attrs
NPP = 300
data["npp"] = xr.full_like(data["npp"], NPP)
data["npp"].attrs = attrs
data["npp"].attrs["units"] = "mg m-2 day-1"

# data = data.sel(latitude=slice(-50, 50))
data

## Sets currents to zero


In [ ]:
data["U"] = data["U"].where(data["U"].isnull(), 0)
data["V"] = data["V"].where(data["U"].isnull(), 0)
data

## Sets temperature to constant value


In [ ]:
TEMPERATURE = 28
data["T"] = data["T"].where(data["T"].isnull(), TEMPERATURE)
data

## Sets primary production to constant value


In [ ]:
NPP = 300
data["npp"] = data["npp"].where(data["npp"].isnull(), NPP)
data

# Dimentions order


In [24]:
data = data.transpose("time", "depth", "latitude", "longitude")

## Create mask


In [ ]:
data["mask"] = xr.full_like(data["T"], 2, dtype=np.int64)
mask = data["mask"][0, 0].drop_vars(["time", "depth"])
mask.latitude.attrs = {
    "standard_name": "latitude",
    "long_name": "latitude",
    "units": "degrees_north",
    "axis": "Y",
}
mask.longitude.attrs = {
    "standard_name": "longitude",
    "long_name": "longitude",
    "units": "degrees_east",
    "axis": "X",
}
mask.attrs = {}
mask.to_netcdf(f"{export_folder}/mask.nc")
xr.load_dataset(f"{export_folder}/mask.nc")

## To float 64


In [26]:
data = data.astype(np.float64)

## Export


In [27]:
for date in data["T"].time.values:
    date_as_string = pd.to_datetime(str(date)).strftime("%Y%m%d")
    data.sel(time=[date])[["U", "V", "T", "npp"]].to_netcdf(f"{export_folder}/data_{date_as_string}.nc")

This shell commands can be run from seapodym-lmtl directory.

```bash
❯ mpirun -mca btl ^openib -np 6 ./build/bin/seapodym-lmtl -P -G D1N1 -V error /Users/ash/Documents/Workspaces/Data/phd/SEAPODYM_LMTL/2024-12-13_-_Constant_fields_no_transport/zpk.tmpl.xml
```

```bash
❯ mpirun -mca btl ^openib -np 6 ./build/bin/seapodym-lmtl -B -G D1N1 -V error /Users/ash/Documents/Workspaces/Data/phd/SEAPODYM_LMTL/2024-12-13_-_Constant_fields_no_transport/zpk.tmpl.xml
```


# TEST -------------


In [ ]:
# MON FORCAGE
data

In [ ]:
# LE FORCAGE ANCIEN
xr.load_dataset(
    "/Users/ash/Documents/Workspaces/Data/configurations_seapodym/lmtl/4_data_integrated/4_2_epi_upmeso_daily/integrated_forcing_20000101.nc"
)

In [ ]:
# MASK ANCIEN
xr.load_dataset(
    "/Users/ash/Documents/Workspaces/Data/configurations_seapodym/lmtl/mask/from_julien/layers_mask_corrected.nc"
)

In [ ]:
# LE RECRUTEMENT ANCIEN
prod = xr.open_mfdataset(
    # "/Users/ash/Documents/Workspaces/Data/phd/SEAPODYM_LMTL/2024-12-13_-_Constant_fields_no_transport/output/*.nc"
    "/Users/ash/Documents/Workspaces/Data/configurations_seapodym/lmtl/output/opti_lambda/*.nc"
)
prod.load()

In [ ]:
# MON RECRUTEMENT
prod = xr.open_mfdataset(
    "/Users/ash/Documents/Workspaces/Data/phd/SEAPODYM_LMTL/2024-12-13_-_Constant_fields_no_transport/output/*.nc"
)
prod.load()